# Sequence Models
Having fun with HMMs, and not only them...!

In [1]:
import sys
sys.path.append('../')
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import lxmls.readers.simple_sequence as ssr 
simple = ssr.SimpleSequence()
print simple.train

[walk/rainy walk/sunny shop/sunny clean/sunny , walk/rainy walk/rainy shop/rainy clean/sunny , walk/sunny shop/sunny shop/sunny clean/sunny ]


In [3]:
print simple.test

[walk/rainy walk/sunny shop/sunny clean/sunny , clean/sunny walk/sunny tennis/sunny walk/sunny ]


In [4]:
for sequence in simple.train.seq_list: 
    print sequence
for sequence in simple.train.seq_list: 
    print sequence.x
for sequence in simple.train.seq_list: 
    print sequence.y

walk/rainy walk/sunny shop/sunny clean/sunny 
walk/rainy walk/rainy shop/rainy clean/sunny 
walk/sunny shop/sunny shop/sunny clean/sunny 
[0, 0, 1, 2]
[0, 0, 1, 2]
[0, 1, 1, 2]
[0, 1, 1, 1]
[0, 0, 0, 1]
[1, 1, 1, 1]


In [5]:
import lxmls.sequences.hmm as hmmc
hmm = hmmc.HMM(simple.x_dict, simple.y_dict) 
hmm.train_supervised(simple.train)
print "Initial Probabilities:\n", hmm.initial_probs
print "Transition Probabilities:\n", hmm.transition_probs
print "Final Probabilities:\n", hmm.final_probs
print "Emission Probabilities:\n", hmm.emission_probs

Initial Probabilities:
[ 0.66666667  0.33333333]
Transition Probabilities:
[[ 0.5    0.   ]
 [ 0.5    0.625]]
Final Probabilities:
[ 0.     0.375]
Emission Probabilities:
[[ 0.75   0.25 ]
 [ 0.25   0.375]
 [ 0.     0.375]
 [ 0.     0.   ]]


### There are non-stationary HMMs where the transition matrix is different for each transition.

In [6]:
initial_scores, transition_scores, final_scores, emission_scores = hmm.compute_scores(simple.train.seq_list[0])
print initial_scores
print transition_scores
print final_scores
print emission_scores

[-0.40546511 -1.09861229]
[[[-0.69314718        -inf]
  [-0.69314718 -0.47000363]]

 [[-0.69314718        -inf]
  [-0.69314718 -0.47000363]]

 [[-0.69314718        -inf]
  [-0.69314718 -0.47000363]]]
[       -inf -0.98082925]
[[-0.28768207 -1.38629436]
 [-0.28768207 -1.38629436]
 [-1.38629436 -0.98082925]
 [       -inf -0.98082925]]


../lxmls/sequences/hmm.py:172: RuntimeWarning: divide by zero encountered in log
  transition_scores[pos-1, :, :] = np.log(self.transition_probs)
../lxmls/sequences/hmm.py:170: RuntimeWarning: divide by zero encountered in log
  emission_scores[pos, :] = np.log(self.emission_probs[sequence.x[pos], :])
../lxmls/sequences/hmm.py:175: RuntimeWarning: divide by zero encountered in log
  final_scores = np.log(self.final_probs)


In [7]:
import numpy as np
a = np.random.rand(10) 
print np.log(sum(np.exp(a)))
print np.log(sum(np.exp(10*a)))
print np.log(sum(np.exp(100*a)))

2.88387476224
9.63393216985
87.7963866123


In [8]:
log_likelihood, forward = hmm.decoder.run_forward(initial_scores, transition_scores, final_scores, emission_scores)
print 'Log-Likelihood =', log_likelihood
log_likelihood, backward = hmm.decoder.run_backward(initial_scores, transition_scores, final_scores, emission_scores)
print 'Log-Likelihood =', log_likelihood

Log-Likelihood = -5.06823232601
Log-Likelihood = -5.06823232601


In [9]:
initial_scores, transition_scores, final_scores, emission_scores = hmm.compute_scores(simple.train.seq_list[0])
state_posteriors, _, _ = hmm.compute_posteriors(initial_scores,
                                                transition_scores, final_scores, emission_scores)

print state_posteriors

[[ 0.95738152  0.04261848]
 [ 0.75281282  0.24718718]
 [ 0.26184794  0.73815206]
 [ 0.          1.        ]]


In [10]:
y_pred = hmm.posterior_decode(simple.test.seq_list[0]) 
print "Prediction test 0:", y_pred
print "Truth test 0:", simple.test.seq_list[0]

Prediction test 0: walk/rainy walk/rainy shop/sunny clean/sunny 
Truth test 0: walk/rainy walk/sunny shop/sunny clean/sunny 


In [11]:
y_pred = hmm.posterior_decode(simple.test.seq_list[1]) 
print "Prediction test 1:", y_pred
print "Truth test 1:", simple.test.seq_list[1]

Prediction test 1: clean/rainy walk/rainy tennis/rainy walk/rainy 
Truth test 1: clean/sunny walk/sunny tennis/sunny walk/sunny 


../lxmls/sequences/sequence_classifier.py:79: RuntimeWarning: invalid value encountered in subtract
  state_posteriors[pos, :] -= log_likelihood
../lxmls/sequences/sequence_classifier.py:92: RuntimeWarning: invalid value encountered in double_scalars
  transition_posteriors[pos, state, prev_state] -= log_likelihood


In [12]:
hmm.train_supervised(simple.train, smoothing=0.1) 
y_pred = hmm.posterior_decode(simple.test.seq_list[0]) 
print "Prediction test 0 with smoothing:", y_pred
print "Truth test 0:", simple.test.seq_list[0]
y_pred = hmm.posterior_decode(simple.test.seq_list[1]) 
print "Prediction test 1 with smoothing:", y_pred
print "Truth test 1:", simple.test.seq_list[1]

Prediction test 0 with smoothing: walk/rainy walk/rainy shop/sunny clean/sunny 
Truth test 0: walk/rainy walk/sunny shop/sunny clean/sunny 
Prediction test 1 with smoothing: clean/sunny walk/sunny tennis/sunny walk/sunny 
Truth test 1: clean/sunny walk/sunny tennis/sunny walk/sunny 


In [13]:
hmm.train_supervised(simple.train, smoothing=0.1)
y_pred, score = hmm.viterbi_decode(simple.test.seq_list[0])
print "Viterbi decoding Prediction test 0 with smoothing:", y_pred, score
print "Truth test 0:", simple.test.seq_list[0]
y_pred, score = hmm.viterbi_decode(simple.test.seq_list[1])
print "Viterbi decoding Prediction test 1 with smoothing:", y_pred, score
print "Truth test 1:", simple.test.seq_list[1]

[[-0.77141589 -2.45413499]
 [-1.838296   -2.87438793]
 [-3.94126804 -3.55180327]
 [-8.44213535 -5.03564784]]
Viterbi decoding Prediction test 0 with smoothing: walk/rainy walk/rainy shop/sunny clean/sunny  -6.02050124698
Truth test 0: walk/rainy walk/sunny shop/sunny clean/sunny 
[[ -4.2054031   -2.06467022]
 [ -5.27228321  -3.93797956]
 [ -9.77315052  -8.85581133]
 [-10.84003063 -10.72912067]]
Viterbi decoding Prediction test 1 with smoothing: clean/sunny walk/sunny tennis/sunny walk/sunny  -11.713974074
Truth test 1: clean/sunny walk/sunny tennis/sunny walk/sunny 
